In [1]:
import numpy as np
import pandas as pd 
import arviz
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, lil_matrix, csr_array

#Import Sparse Barrier Random Walks
from polytopewalk.sparse import SparseDikinWalk, SparseVaidyaWalk, SparseJohnWalk

#Import Dense Barrier Random Walks
from polytopewalk.dense import DikinWalk, VaidyaWalk, JohnWalk

def generate_simplex(d):
    return np.array([1/d] * d), np.array([[1] * d]), np.array([1]), d

In [2]:
#Create 5d simplex polytope for sparse testing
x1, A1, b1, k1 = generate_simplex(5)

k = 5
cov_simplex = np.full((k,k), (-1.0 / (k*k)) / (k + 1.0))
for i in range(k):
    cov_simplex[i,i] = (1 - (1.0 / (k))) * (1.0/k) / (k + 1.0)

In [3]:
#Run 5 chains on 100,000 iterations with burnin parameter of 1000 and thin parameter of 10
nchains = 5
niter = 100_000
burn = 1000
thin = 10

seed = [1000, 2000, 3000, 4000, 5000]
sparse_dikin = SparseDikinWalk(r = 0.5)
sparse_vaidya = SparseVaidyaWalk(r = 0.7)
sparse_john = SparseJohnWalk(r = 0.7)

walks_all_dikin = np.empty([nchains, (niter-burn)//thin, k])
walks_all_vaidya = np.empty([nchains, (niter-burn)//thin, k])
walks_all_john = np.empty([nchains, (niter-burn)//thin, k])
for c in range(nchains):
    walks_all_dikin[c, :] = sparse_dikin.generateCompleteWalk(niter, x1, A1, b1, k1, burnin = burn, thin = thin, seed = seed[c])
    walks_all_vaidya[c, :] = sparse_vaidya.generateCompleteWalk(niter, x1, A1, b1, k1, burnin = burn, thin = thin, seed = seed[c])
    walks_all_john[c, :] = sparse_john.generateCompleteWalk(niter, x1, A1, b1, k1, burnin = burn, thin = thin, seed = seed[c])

In [4]:
#Compute rhat to see if there is convergence to a distribution

walks_dikin = arviz.convert_to_dataset(walks_all_dikin)
max_rhat_dikin = arviz.rhat(walks_dikin).to_array().values.max().item()

walks_vaidya = arviz.convert_to_dataset(walks_all_vaidya)
max_rhat_vaidya = arviz.rhat(walks_vaidya).to_array().values.max().item()

walks_john = arviz.convert_to_dataset(walks_all_john)
max_rhat_john = arviz.rhat(walks_john).to_array().values.max().item()

max_rhat_dikin, max_rhat_vaidya, max_rhat_john

(1.0070817737224627, 1.0051494205272644, 1.0071241574972754)

In [5]:
#Computing the difference between empirical mean/covariance and known mean/covariance

walks_all_dikin_np = np.concatenate([walks_all_dikin[i,:,:] for i in range(nchains)])
dikin_cov = np.cov(walks_all_dikin_np, rowvar = False)

walks_all_vaidya_np = np.concatenate([walks_all_vaidya[i,:,:] for i in range(nchains)])
vaidya_cov = np.cov(walks_all_vaidya_np, rowvar = False)

walks_all_john_np = np.concatenate([walks_all_john[i,:,:] for i in range(nchains)])
john_cov = np.cov(walks_all_john_np, rowvar = False)

dikin_cov_diff = np.linalg.norm(dikin_cov - cov_simplex, ord='fro')
vaidya_cov_diff = np.linalg.norm(vaidya_cov - cov_simplex, ord='fro')
john_cov_diff = np.linalg.norm(john_cov - cov_simplex, ord='fro')

dikin_mean_diff = np.linalg.norm(walks_all_dikin_np.mean(axis = 0) - [0.2] * 5)
vaidya_mean_diff = np.linalg.norm(walks_all_vaidya_np.mean(axis = 0) - [0.2] * 5)
john_mean_diff = np.linalg.norm(walks_all_john_np.mean(axis = 0) - [0.2] * 5)

print(dikin_cov_diff, vaidya_cov_diff, john_cov_diff)
print(dikin_mean_diff, vaidya_mean_diff, john_mean_diff)

0.0012444532007629282 0.0017318853821796472 0.0013303631926015384
0.005223340170087404 0.0046792622976124035 0.005903213095929759


In [6]:
#Create 3d hypercube polytope for dense testing
x2 = np.array([0] * 3)
A2 = np.array([[1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0], [0, 0, 1], [0, 0, -1]])
b2 = np.array([1] * 6)
cov_hc = 1/3 * np.eye(3)

In [7]:
#Run 5 chains on 300,000 iterations with burnin parameter of 1_000 and thin parameter of 10
nchains = 5
niter = 300_000
burn = 1_000
thin = 10

seed = [1000, 2000, 3000, 4000, 5000]
dikin = DikinWalk(r = 0.8)
vaidya = VaidyaWalk(r = 0.9)
john = JohnWalk(r = 0.9)

walks_all_dikin = np.empty([nchains, (niter-burn)//thin, 3])
walks_all_vaidya = np.empty([nchains, (niter-burn)//thin, 3])
walks_all_john = np.empty([nchains, (niter-burn)//thin, 3])
for c in range(nchains):
    walks_all_dikin[c, :] = dikin.generateCompleteWalk(niter, x2, A2, b2, burnin = burn, thin = thin, seed = seed[c])
    walks_all_vaidya[c, :] = vaidya.generateCompleteWalk(niter, x2, A2, b2, burnin = burn, thin = thin, seed = seed[c])
    walks_all_john[c, :] = john.generateCompleteWalk(niter, x2, A2, b2, burnin = burn, thin = thin, seed = seed[c])

In [8]:
#Compute rhat to see if there is convergence to a distribution

walks_dikin = arviz.convert_to_dataset(walks_all_dikin)
max_rhat_dikin = arviz.rhat(walks_dikin).to_array().values.max().item()

walks_vaidya = arviz.convert_to_dataset(walks_all_vaidya)
max_rhat_vaidya = arviz.rhat(walks_vaidya).to_array().values.max().item()

walks_john = arviz.convert_to_dataset(walks_all_john)
max_rhat_john = arviz.rhat(walks_john).to_array().values.max().item()

max_rhat_dikin, max_rhat_vaidya, max_rhat_john

(1.0008391082946915, 1.0011157493535179, 1.0010260939235482)

In [9]:
#Computing the difference between empirical mean/covariance and known mean/covariance

walks_all_dikin_np = np.concatenate([walks_all_dikin[i,:,:] for i in range(nchains)])
dikin_cov = np.cov(walks_all_dikin_np, rowvar = False)

walks_all_vaidya_np = np.concatenate([walks_all_vaidya[i,:,:] for i in range(nchains)])
vaidya_cov = np.cov(walks_all_vaidya_np, rowvar = False)

walks_all_john_np = np.concatenate([walks_all_john[i,:,:] for i in range(nchains)])
john_cov = np.cov(walks_all_john_np, rowvar = False)

dikin_cov_diff = np.linalg.norm(dikin_cov - cov_hc, ord='fro')
vaidya_cov_diff = np.linalg.norm(vaidya_cov - cov_hc, ord='fro')
john_cov_diff = np.linalg.norm(john_cov - cov_hc, ord='fro')

dikin_mean_diff = np.linalg.norm(walks_all_dikin_np.mean(axis = 0) - [0.0] * 3)
vaidya_mean_diff = np.linalg.norm(walks_all_vaidya_np.mean(axis = 0) - [0.0] * 3)
john_mean_diff = np.linalg.norm(walks_all_john_np.mean(axis = 0) - [0.0] * 3)

print(dikin_cov_diff, vaidya_cov_diff, john_cov_diff)
print(dikin_mean_diff, vaidya_mean_diff, john_mean_diff)

0.006754593574308272 0.007232981472577084 0.007729710269585108
0.006344973743305176 0.011318646710829806 0.00969524865630466
